# Step 2: Pre-processing - Calculating annual Mean 📅

The goal is to calculate the **yearly mean** of the variables from the **Baltic Sea Wave Hindcast** dataset, which is available only in **hourly resolution**. Each day has a separate file, and each file is around **200MB** in size. If I wanted to download the entire dataset and calculate the mean, I would need **73GB** of storage! Here's the math: (*365 * 200 = 73,000 MB = 73 GB*)... 

Since it is quite too big, I will use a technique often employed in survey sampling. Instead of downloading the entire dataset, I will download a **random sample** of the data and calculate the mean based on this sample. To make it more manageable, I’ll assume I download **30%** of the dataset, which means downloading **109 files** (*365 * 0.30 = 109*).

#### 📚 Required Libraries
To carry out this process, you'll need the following libraries:
- **`pandas`**: For reading, manipulating, and analyzing the data. 
- **`random`**: For generating random samples from the data. 
- **`copernicusmarine`**: To dowload the dataset with the Copernicus marine API.
- **`xarray`**: To open the dowloaded datasets.
- **`geopandas`**: To open the geometry of the Living Lab West.
- **`numpy`**: To obtain the converty the Living Lab West geometry as a simple rectangle, its bounding box.
- **`time`**: To ensure files are well deleted.
- **`os`**: For ensuring the path exists.
- **`gc`**: To help in the deleting precess.

### 🛠️ Steps:
1. **Generate a Random List of Dates**: Create a list of random dates to represent 10% of the total dataset.
2. **Download the Files**: Using the list of random dates, download the corresponding files from the API. 
3. **Spatially filtering each daily dataset by the bounding box of the Living Lab Geometry**: Use a spatial filter to restrict the data to the region of interest. I used the bounding box of the `livinglab_geometry` rather than the livinglab_geometry to avoid any loss of data. Spatially filtering is useless because of resampling. See the **Ressampling** Notebook.
2. **Calculating the annual mean of the filtered dataset**: By combining the datasets, calculate the yearly mean.

This method significantly reduces the amount of data needed while still providing a statistically valid approximation.

______


### 🛠️ Generating a Random List of Dates 

Let’s start by generating a random list of **328 dates**. This list will determine which files to download and will be the foundation for calculating the mean based on a representative sample of the data.

In [ ]:
import pandas as pd
import random
import os

In [3]:
# Generate the full list of dates from January 1, 2015 to December 31, 2023
start_date = "2023-01-01"
end_date = "2023-12-31"

# Generate the date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Randomly sample 328 dates from this range
sampled_dates = random.sample(list(date_range), 109)

sampled_dates.sort()

We can now save our random dates in a csv file. Once I saved it, I make sure not to overwrite the file.

In [ ]:
file_path = 'data/sampled_dates.csv'
if not os.path.exists(file_path):
    pd.Series(sampled_dates).to_csv(file_path, index=False)
else:
    print('The file already exists. Delete it if you want to save a new one.')

### 🛠️ Dowloading the files

I don't want to dowload this 109 files one by one. I can use the API of Copernicus Marine Data Store. It is possible to create a list of files or dowload the files one by one. You can find more informations on the [Copernicus Marine API page](https://help.marine.copernicus.eu/en/articles/8286883-copernicus-marine-toolbox-api-get-original-files#h_4cb9923744).

⚠️ The dowloaded files are not saved in the GitHub repositary because it was too heavy (22GB). However, the spatial filtered annual mean version is saved.

In [6]:
import copernicusmarine as cm

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


To dowload a dataset of the Copernicus Marine API, you have to use its ID. You can found its ID in the Data access page, in grey on the Dataset column. You can click on "📑​" to copy it. It will display a message : "Copied dataset ID to your clipboard".

In [7]:
datasetID = 'cmems_mod_bal_wav_my_PT1H-i'
sampled_dates = pd.read_csv('data/sampled_dates.csv', parse_dates=[0]).iloc[:, 0].tolist()

You need to create an account to dowload the dataset. Below is the method to dowload a specific file. (I have dowloaded one by one).

In [ ]:
i=0
for date in sampled_dates:
    i += 1
    year = date.year
    month = date.month 
    if month < 10:
        month = f'0{month}'             
    day = date.day
    if day < 10:
        day = f'0{day}'
    cm.get(
        username = 'your_username',
        password = 'your_password',
        dataset_id  = datasetID,
        filter=f'*{year}{month}{day}*'
        )
    print(f'Dowloading {i} / 109 files for {year}-{month}-{day}')

INFO - 2025-08-07T12:18:51Z - Selected dataset version: "202411"
INFO - 2025-08-07T12:18:51Z - Selected dataset part: "default"
INFO - 2025-08-07T12:18:51Z - Listing files on remote server...
17it [00:05,  3.27it/s]


Dowloading 1 / 109 files for 2023-01-02


INFO - 2025-08-07T12:19:11Z - Selected dataset version: "202411"
INFO - 2025-08-07T12:19:11Z - Selected dataset part: "default"
INFO - 2025-08-07T12:19:11Z - Listing files on remote server...
17it [00:05,  2.90it/s]


Dowloading 2 / 109 files for 2023-01-03


INFO - 2025-08-07T12:19:33Z - Selected dataset version: "202411"
INFO - 2025-08-07T12:19:33Z - Selected dataset part: "default"
INFO - 2025-08-07T12:19:34Z - Listing files on remote server...
17it [00:06,  2.72it/s]


...

Dowloading 107 / 109 files for 2023-12-18


INFO - 2025-08-07T13:01:45Z - Selected dataset version: "202411"
INFO - 2025-08-07T13:01:45Z - Selected dataset part: "default"
INFO - 2025-08-07T13:01:45Z - Listing files on remote server...
17it [00:05,  2.89it/s]


Dowloading 108 / 109 files for 2023-12-24


INFO - 2025-08-07T13:02:05Z - Selected dataset version: "202411"
INFO - 2025-08-07T13:02:05Z - Selected dataset part: "default"
INFO - 2025-08-07T13:02:06Z - Listing files on remote server...
17it [00:05,  3.24it/s]

Dowloading 109 / 109 files for 2023-12-30


### 🛠️ Spatially filtering each daily dataset by the bounding box of the Living Lab Geometry

I need to filter spatially the dataset before the calcul of the annual mean because I don't want to calcul the values of the mean for all the Baltic Sea. However, using the Living Lab West mask can create issues at the border of the geometry and cause a loss of data at the border... because I am working in shalow water, the border data are very important ! I will create and use the boundingbox of the living lab geometry as a mask.

I will merge the datasets per year and caculate the mean.

In [ ]:
import xarray as xr
import geopandas as gpd
import numpy as np
import time
import matplotlib.pyplot as plt
import gc

In [11]:
livinglab_geometry = gpd.read_file("data/livinglab_west.json")

In [13]:
i = 0
for date in sampled_dates:
    i += 1
    year = date.year
    month = date.month
    if month < 10:
        month = f'0{month}'             
    day = date.day
    if day < 10:
        day = f'0{day}'
    input_file_name = f'CMEMS_BAL_WAV_MY_{year}{month}{day}01.nc'
    output_file_name = f'bounding_box_filtered_{input_file_name}'
    input_file = f'data/{year}/{month}/{input_file_name}'
    output_file = f'data/{year}/{month}/{output_file_name}'

    ds = xr.open_dataset(input_file)
    lat = ds['lat'].values
    lon = ds['lon'].values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    coords = np.column_stack((lon_grid.ravel(), lat_grid.ravel()))
    # Get the bounding box of the livinglab_geometry
    minx, miny, maxx, maxy = livinglab_geometry.total_bounds
    # Create a mask using the bounding box
    mask = ((lat_grid >= miny) & (lat_grid <= maxy) & (lon_grid >= minx) & (lon_grid <= maxx))
    mask_da = xr.DataArray(mask, coords=[ds['lat'], ds['lon']], dims=["lat", "lon"])
    filtered_data = ds.where(mask_da, drop=True) 
    
    try:
        filtered_data.to_netcdf(output_file)
    finally:
        ds.close()
        filtered_data.close()
        del ds
        del filtered_data
        gc.collect()
        time.sleep(1)

    # Try to delete the file, retrying if necessary
    for attempt in range(5):
        try:
            if os.path.exists(output_file):
                os.remove(input_file)
            break
        except PermissionError:
            time.sleep(1)
    print(f'File {i} / 109 processed: {output_file_name}')

File 1 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023010201.nc
File 2 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023010301.nc
File 3 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023010701.nc
File 4 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023011301.nc
File 5 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023011401.nc
File 6 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023011701.nc
File 7 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023011801.nc
File 8 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023012601.nc
File 9 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023012701.nc
File 10 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023012801.nc
File 11 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023012901.nc
File 12 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2023013101.nc
File 13 / 109 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_202302060

### 🛠️ Calculating the annual mean of the filtered dataset

In [15]:
# Filter the sampled_dates list for the specified year
files = [f'data/{date.year}/{date.month:02d}/bounding_box_filtered_CMEMS_BAL_WAV_MY_{date.year}{date.month:02d}{date.day:02d}01.nc' for date in sampled_dates]
# Open the datasets and concatenate them along the time dimension
datasets = [xr.open_dataset(file) for file in files]
# Concatenate the datasets along the 'time' dimension
combined_dataset = xr.concat(datasets, dim='time')
# Calculate the mean for each variable across time for the year
yearly_mean = combined_dataset.mean(dim='time')
yearly_mean

<xarray.Dataset> Size: 2MB
Dimensions:    (lat: 227, lon: 109)
Coordinates:
  * lat        (lat) float32 908B 55.34 55.36 55.37 55.39 ... 59.07 59.09 59.11
  * lon        (lon) float32 436B 10.04 10.07 10.1 10.12 ... 12.99 13.01 13.04
Data variables: (12/19)
    VHM0       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VTPK       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VTM10      (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VTM02      (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VMDR       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VHM0_WW    (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    ...         ...
    VTM01_SW2  (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VMDR_SW2   (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VSDX       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VSDY       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VMXL       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VCMX       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan

In [17]:
# Save the file
yearly_mean.to_netcdf(f'data/CMEMS_BAL_WAV_MY-AnnualMeans-2023.nc')